<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#quick-start" data-toc-modified-id="quick-start-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>quick start</a></span><ul class="toc-item"><li><span><a href="#Resources" data-toc-modified-id="Resources-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Resources</a></span></li></ul></li><li><span><a href="#FULL-MODEL" data-toc-modified-id="FULL-MODEL-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>FULL MODEL</a></span><ul class="toc-item"><li><span><a href="#Packages" data-toc-modified-id="Packages-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Packages</a></span><ul class="toc-item"><li><span><a href="#Qnetwork" data-toc-modified-id="Qnetwork-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Qnetwork</a></span></li></ul></li><li><span><a href="#POMDP-RNN-Game" data-toc-modified-id="POMDP-RNN-Game-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>POMDP RNN Game</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Results</a></span><ul class="toc-item"><li><span><a href="#Standard-setting：-grid-=--(3,7)，-holes-=-0" data-toc-modified-id="Standard-setting：-grid-=--(3,7)，-holes-=-0-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Standard setting： grid =  (3,7)， holes = 0</a></span></li><li><span><a href="#Model-Tranining" data-toc-modified-id="Model-Tranining-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Model Tranining</a></span></li><li><span><a href="#decoding-vs-performance" data-toc-modified-id="decoding-vs-performance-2.3.3"><span class="toc-item-num">2.3.3&nbsp;&nbsp;</span>decoding vs performance</a></span></li><li><span><a href="#learning-rate-vs-performance" data-toc-modified-id="learning-rate-vs-performance-2.3.4"><span class="toc-item-num">2.3.4&nbsp;&nbsp;</span>learning rate vs performance</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-2.3.5"><span class="toc-item-num">2.3.5&nbsp;&nbsp;</span>Summary</a></span></li><li><span><a href="#Anlytic-part-,-check-the-behaviour-correspond-to-each-decoding-level-and-explain-why" data-toc-modified-id="Anlytic-part-,-check-the-behaviour-correspond-to-each-decoding-level-and-explain-why-2.3.6"><span class="toc-item-num">2.3.6&nbsp;&nbsp;</span>Anlytic part , check the behaviour correspond to each decoding level and explain why</a></span></li></ul></li><li><span><a href="#PCA" data-toc-modified-id="PCA-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>PCA</a></span></li></ul></li></ul></div>

# quick start 

Test what is the decoding change for single size training 

## Resources

* [*The* Reinforcement learning book from Sutton & Barto](http://incompleteideas.net/sutton/book/the-book-2nd.html)
* [The REINFORCE paper from Ronald J. Williams (1992)](http://www-anw.cs.umass.edu/~barto/courses/cs687/williams92simple.pdf)


# FULL MODEL

## Packages

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from itertools import count
import random

# write a new algorithm with a pointer or attention on top of th two

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable
from torch.nn import init
from torch.nn import DataParallel

import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML

import POMDPgame_bars
from POMDPgame_bars import*

import POMDPgame_basic
from POMDPgame_basic import*

import POMDPgame_holes
from POMDPgame_holes import*


import RNN
from RNN import *

import navigation2
from navigation2 import*

import Nets
from Nets import*

%matplotlib inline

### Qnetwork

To select actions we take maximum of Q value, corresponding to certain move.

For the liquid state approach to work, you need a lot of neurons as surplus or enough hidden to hidden connectivity to make it have an effect.

##  POMDP RNN Game

In this game , we use a new reward function determined by game, if the agent achieves the goal before 50, reward is 1. If time pass 50 reward is 0.5, once time pass 100 agent gets a reward of -0.5 .  Practically, this is found to be easier to learn than the rewards as a continous function of time.  Tf the agent learns to search in a efficient way, the largest possible way for search is to firstly arrive at corner then goes to the goal, which, takes about 50 steps, it is reasonble to make 50 and 100 as milestone thing.  Also in principe as the game doesn't have a timer , it is not if it can use a reward as funtion of time

3 condition for ending , when pass time limit, game over

For weight update, it seems to be better do it after episode, as it makes non-sense evaluate strategy during episode, but a the end. Also, it is much quicker. 

A programming of MDP here, hidden state is as state of enviroment

Think of how to make this traning stable, adding exploration noise , intenral noise or environmental variability like multiple mazes  

In [6]:
# moving bar train
trial = 399
tasks = ['scale']
iterations = [4]
# iterations = [1, 1, 1, 1, 1, 1, 1]
Scores = np.zeros((1, 5))
i = 0
# weight_read = 'weights_' + 'scale' + '/rnn_1515tanh512_checkpoint{}_{}'.format(399, 5)
for iters, task in zip(iterations, tasks):
        Task = MultipleTasks(task=task, weight_write='weights_cpu/rnn_1515tanh512_checkpoint{}'.format(0) \
                         , noise=0.0, weight1='weights_cpu_pos/rnn_1515tanh512_checkpoint399',
                         weight2='weights_cpu_mem/rnn_1515tanh512_checkpoint49')
        for k in range(2):
            weight_read = 'weights_' + task + '/rnn_1515tanh512_checkpoint{}_{}'.format(trial, 6 - k)
            if task == 'scale':
                score = Test(task, Task.game, weight = weight_read, size = 50, test_size = 2, limit_set = 4, step = 3)
            elif (task == 'scale_x') or (task == 'scale_y'):
                score = Test(task, Task.game, weight = weight_read, test_size = 1, limit_set = 4)
            else:
                score = Test(task, Task.game, weight = weight_read, limit_set = 2)
            Scores[i, k]= score
        i += 1
np.save('Scores_scale', Scores)

task <class 'POMDPgame_scale.GameScale'>
scale


/home/yons/.local/lib/python3.6/site-packages/torch/tensor.py:293: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


(27, 14)
(27, 39)
scale performance 0.6062186131026897
(27, 14)
(27, 39)
scale performance -0.19379810723870125


In [ ]:
# moving bar train
trial = 399
tasks = ['scale_y']
iterations = [4]
# iterations = [1, 1, 1, 1, 1, 1, 1]
Scores = np.zeros((1, 5))
i = 0
# weight_read = 'weights_' + 'scale' + '/rnn_1515tanh512_checkpoint{}_{}'.format(399, 5)
for iters, task in zip(iterations, tasks):
        Task = MultipleTasks(task=task, weight_write='weights_cpu/rnn_1515tanh512_checkpoint{}'.format(0) \
                         , noise=0.0, weight1='weights_cpu_pos/rnn_1515tanh512_checkpoint399',
                         weight2='weights_cpu_mem/rnn_1515tanh512_checkpoint49')
        for k in range(5):
            weight_read = 'weights_' + task + '/rnn_1515tanh512_checkpoint{}_{}'.format(trial, 9 - k)
            if task == 'scale':
                score = Test(task, Task.game, weight = weight_read, size = 50, test_size = 2, limit_set = 4, step = 3)
            elif (task == 'scale_x') or (task == 'scale_y'):
                score = Test(task, Task.game, weight = weight_read, test_size = 1, limit_set = 4)
            else:
                score = Test(task, Task.game, weight = weight_read, limit_set = 2)
            Scores[i, k]= score
        i += 1
np.save('Scores_scale', Scores)

task <class 'POMDPgame_scale_y.GameScale_y'>
scale_y


/home/yons/.local/lib/python3.6/site-packages/torch/tensor.py:293: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


scale_y performance 0.3649136089421072
scale_y performance 0.02388265837024215
scale_y performance 0.8186569467317381


In [3]:
#### moving bar train
trial = 399
tasks = ['basic', 'hole', 'bar', 'scale', 'scale_x', 'scale_y', 'implicit']
iterations = [9, 29, 29, 9, 29, 29, 29]
# iterations = [1, 1, 1, 1, 1, 1, 1]
Scores = np.zeros((7, 5))
i = 0
# weight_read = 'weights_' + 'scale' + '/rnn_1515tanh512_checkpoint{}_{}'.format(399, 5)
for iters, task in zip(iterations, tasks):
        Task = MultipleTasks(task=task, weight_write='weights_cpu/rnn_1515tanh512_checkpoint{}'.format(0) \
                         , noise=0.0, weight1='weights_cpu_pos/rnn_1515tanh512_checkpoint399',
                         weight2='weights_cpu_mem/rnn_1515tanh512_checkpoint49')
        for k in range(5):
            weight_read = 'weights_' + task + '/rnn_1515tanh512_checkpoint{}_{}'.format(trial,  iters - k)
            if task == 'scale':
                score = Test(task, Task.game, weight = weight_read, size = 50, test_size = 2, limit_set = 4, step = 3)
            elif (task == 'scale_x') or (task == 'scale_y'):
                score = Test(task, Task.game, weight = weight_read, test_size = 1, limit_set = 4)
            else:
                score = Test(task, Task.game, weight = weight_read)
            Scores[i, k]= score
        i += 1
np.save('Scores_modules', Scores)

task <class 'POMDPgame_basic.GameBasic'>
basic


/home/yons/.local/lib/python3.6/site-packages/torch/tensor.py:293: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


basic performance 0.9031684078176023
basic performance 0.9292655760835254
basic performance 0.9204456103268506
basic performance 0.9565360776899936
basic performance 0.9438141953017409
task <class 'POMDPgame_holes.GameHole'>
hole
hole performance 0.6746172556586527
hole performance 0.5689470329838906
hole performance 0.6639240090923737
hole performance 0.6890389812398598
hole performance 0.7242161812663229
task <class 'POMDPgame_bars.GameBar'>
bar
bar performance 0.6801895145466739
bar performance 0.6627369944749971
bar performance 0.6871498169591541
bar performance 0.6992439398019161
bar performance 0.647436320141778
task <class 'POMDPgame_scale.GameScale'>
scale
(27, 14)
(27, 39)
scale performance -0.7352362239514121
(27, 14)
(27, 39)
scale performance 0.32732565036318806
(27, 14)
(27, 39)
scale performance -0.06033695633960101
(27, 14)
(27, 39)
scale performance 0.5983250657577792
(27, 14)
(27, 39)
scale performance -0.22192771082991034
task <class 'POMDPgame_scale_x.GameScale_x'>
s

In [2]:
#### moving bar train
trial = 399
tasks = ['basic', 'hole', 'bar', 'scale', 'scale_x', 'scale_y', 'implicit']
iterations = [9, 29, 29, 9, 29, 29, 29]
# iterations = [1, 1, 1, 1, 1, 1, 1]
Scores = np.zeros((7, 5))
i = 0
# weight_read = 'weights_' + 'scale' + '/rnn_1515tanh512_checkpoint{}_{}'.format(399, 5)
for iters, task in zip(iterations, tasks):
        Task = MultipleTasks(task=task, weight_write='weights_cpu/rnn_1515tanh512_checkpoint{}'.format(0) \
                         , noise=0.0, weight1='weights_cpu_pos/rnn_1515tanh512_checkpoint399',
                         weight2='weights_cpu_mem/rnn_1515tanh512_checkpoint49')
        for k in range(5):
            weight_read = 'weights_' + task + '/rnn_1515tanh512_checkpoint{}_{}'.format(trial,  iters - k)
            if task == 'scale':
                score = Test(task, Task.game, weight = weight_read, size = 50, test_size = 2, limit_set = 4, step = 3)
            elif (task == 'scale_x') or (task == 'scale_y'):
                score = Test(task, Task.game, weight = weight_read, test_size = 1, limit_set = 4)
            else:
                score = Test(task, Task.game, weight = weight_read)
            Scores[i, k]= score
        i += 1
np.save('Scores_modules', Scores)

task <class 'POMDPgame_basic.GameBasic'>
basic


/home/yons/.local/lib/python3.6/site-packages/torch/tensor.py:293: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


basic performance 0.9339615118069065
basic performance 0.9304340718755142
basic performance 0.9532086957925652
basic performance 0.9628908475956623
basic performance 0.9378663016288779
task <class 'POMDPgame_holes.GameHole'>
hole
hole performance 0.7402882456923582
hole performance 0.6914422832109195
hole performance 0.6996684938740919
hole performance 0.7073232140178183
hole performance 0.7583449039148773
task <class 'POMDPgame_bars.GameBar'>
bar
bar performance 0.7525221616876329
bar performance 0.7457533812179759
bar performance 0.7209314194122264
bar performance 0.7555190514046531
bar performance 0.7228026752236519
task <class 'POMDPgame_scale.GameScale'>
scale
(27, 14)
(27, 39)
scale performance -0.7246805346780507
(27, 14)
(27, 39)
scale performance 0.313059165607429
(27, 14)
(27, 39)
scale performance -0.06168218106440998
(27, 14)
(27, 39)
scale performance 0.6180081347017202
(27, 14)
(27, 39)
scale performance -0.21070521449411628
task <class 'POMDPgame_scale_x.GameScale_x'>
sc